<a href="https://colab.research.google.com/github/sidheshsahu/Custom_NER/blob/main/CUSTOM_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Custom_NER"


/content/drive/MyDrive/Custom_NER


In [ ]:
!pip install -U spacy
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json


spacy.__version__

'3.7.6'

In [ ]:

cv_data = json.load(open('/content/drive/MyDrive/Custom_NER/annotations_dataset/annotations _semantic.json','r'))


len(cv_data)


cv_data

[['Rajesh Kumar filed a lawsuit against the National Bank for fraud, but later relaxed on the bank of the Ganga river.\r',
  {'entities': [[0, 12, 'PERSON'], [41, 54, 'ORG'], [103, 114, 'LOC']]}],
 ['Mehta & Co. Advocates represented Infosys in court, while Ananya enjoyed an apple at her desk.\r',
  {'entities': [[0, 21, 'ORG'], [34, 41, 'ORG'], [58, 64, 'PERSON']]}],
 ['Aarav received an email from r.menon@legalfirm.in about his will, but didn’t check it before drafting his will for his property in Delhi.\r',
  {'entities': [[0, 5, 'PERSON'], [29, 49, 'EMAIL'], [131, 136, 'GPE']]}],
 ['Justice Nisha Verma ruled in favor of the Trust, but struggled to trust the evidence presented in court.\r',
  {'entities': [[0, 19, 'PERSON'], [38, 47, 'ORG']]}],
 ['Priya was billed for legal consultation charges, but was later charged with contempt for missing a court date.\r',
  {'entities': [[0, 5, 'PERSON']]}],
 ['Dr. Suman Iyer called her patient using her clinic’s number, +91 9866754002, but her

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/Custom_NER/config/base_config.cfg /content/drive/MyDrive/Custom_NER/config/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Custom_NER/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()


  #Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db


In [ ]:

from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.2)


len(train), len(test)


file = open('/content/drive/MyDrive/Custom_NER/trained_models/train_file.txt','w')


db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/Custom_NER/trained_models/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/Custom_NER/trained_models/test_data.spacy')


file.close()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 57/57 [00:00<00:00, 969.56it/s]


In [ ]:
!python -m spacy train /content/drive/MyDrive/Custom_NER/config/config.cfg  --output /content/drive/MyDrive/Custom_NER/trained_models/output  --paths.train /content/drive/MyDrive/Custom_NER/trained_models/train_data.spacy  --paths.dev /content/drive/MyDrive/Custom_NER/trained_models/test_data.spacy --gpu-id 0

✔ Created output directory:
/content/drive/MyDrive/Custom_NER/trained_models/output
ℹ Saving to output directory:
/content/drive/MyDrive/Custom_NER/trained_models/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new down

In [3]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have

In [ ]:
# import spacy
# import spacy_transformers
# # Load the trained spaCy NER model from the specified path
# nlp = spacy.load('/content/drive/MyDrive/Custom_NER/trained_models/output/model-best')

# # Define a function to classify NER in a given sentence
# def classify_ner(sentence):
#     # Process the sentence using the trained spaCy model
#     doc = nlp(sentence)

#     # Extract entities and their labels and create a formatted output string
#     formatted_output = "\n".join([f"{ent.text}: {ent.label_}" for ent in doc.ents])

#     return formatted_output
# sentence="Ms. Shreya Rao made a transfer of ₹95,000 on 9th October 2023 at 02:15 PM. Her account number is 4561237896523 with HDFC Bank (IFSC: HDFC0006547). Her PAN is FGHIJ5678K, and Aadhaar number is 3456 7890 2345. Reach her at +91 9123456789 or shreya.rao@example.com."

# #Ms. Shreya Rao made a transfer of ₹95,000 on 9th October 2023 at 02:15 PM. Her account number is 4561237896523 with HDFC Bank (IFSC: HDFC0006547). Her PAN is FGHIJ5678K, and Aadhaar number is 3456 7890 2345. Reach her at +91 9123456789 or shreya.rao@example.com.
# formatted_entities = classify_ner(sentence)

# print("Entity mapping:\n", formatted_entities)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.7.6 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed du

Entity mapping:
 Ms. Shreya Rao: PERSON
₹95,000: MONEY
9th October 2023: DATE
02:15 PM.: TIME
4561237896523: ACCOUNT
HDFC Bank: ORG
HDFC0006547: IFSC
FGHIJ5678K: PAN
3456 7890 2345.: ACN
+91 9123456789: PHONE
shreya.rao@example.com: EMAIL


In [4]:
import spacy
from spacy import displacy


nlp = spacy.load('/content/drive/MyDrive/Custom_NER/trained_models/output/model-best')


colors = {

    "PERSON": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",  # Purple to Pink gradient
    "ORG": "#ffd700",  # Gold
    "GPE": "#7fffd4",  # Aquamarine
    "DATE": "#ff69b4",  # Hot Pink (Legal)
    "EMAIL": "#4682b4",  # Steel Blue
    "PHONE": "#32cd32",  # Lime Green
    "MEDICATION": "#ba55d3",  # Medium Orchid (Health)
    "HEALTH": "#ba55d3",  # Medium Orchid (Health)
    "AGE": "#ff69b4",  # Hot Pink (Legal)
    "TIME": "#ff69b4",  # Hot Pink (Legal)
    "ACCOUNT": "#32cd32",  # Lime Green (Banks)
    "IFSC": "#32cd32",  # Lime Green (Banks)
    "PAN": "#ff69b4",  # Hot Pink (Legal)
    "ACN": "#32cd32",  # Lime Green (Banks)
    "MONEY": "#32cd32"  # Lime Green (Banks)
}



options = {
    "ents": ["PERSON", "ORG", "GPE", "DATE", "EMAIL", "PHONE", "MEDICATION", "HEALTH","TIME","ACCOUNT","IFSC","PAN","ACN","AGE","MONEY"],  # Your entity labels
    "colors": colors
}


def classify_and_visualize_ner(sentence):

    doc = nlp(sentence)

    displacy.render(doc, style="ent", options=options, jupyter=True)

sentence = "Ms. Shreya Rao made a transfer of ₹95,000 on 9th October 2023 at 02:15 PM. Her account number is 4561237896523 with HDFC Bank (IFSC: HDFC0006547). Her PAN is FGHIJ5678K, and Aadhaar number is 3456 7890 2345. Reach her at +91 9123456789 or shreya.rao@example.com."


classify_and_visualize_ner(sentence)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.7.6 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/gen

In [5]:
import spacy
from spacy import displacy


nlp = spacy.load('/content/drive/MyDrive/Custom_NER/trained_models/output/model-best')


colors = {

    "PERSON": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",  # Purple to Pink gradient
    "ORG": "#ffd700",  # Gold
    "GPE": "#7fffd4",  # Aquamarine
    "DATE": "#ff69b4",  # Hot Pink (Legal)
    "EMAIL": "#4682b4",  # Steel Blue
    "PHONE": "#32cd32",  # Lime Green
    "MEDICATION": "#ba55d3",  # Medium Orchid (Health)
    "HEALTH": "#ba55d3",  # Medium Orchid (Health)
    "AGE": "#ff69b4",  # Hot Pink (Legal)
    "TIME": "#ff69b4",  # Hot Pink (Legal)
    "ACCOUNT": "#32cd32",  # Lime Green (Banks)
    "IFSC": "#32cd32",  # Lime Green (Banks)
    "PAN": "#ff69b4",  # Hot Pink (Legal)
    "ACN": "#32cd32",  # Lime Green (Banks)
    "MONEY": "#32cd32"  # Lime Green (Banks)
}



options = {
    "ents": ["PERSON", "ORG", "GPE", "DATE", "EMAIL", "PHONE", "MEDICATION", "HEALTH","TIME","ACCOUNT","IFSC","PAN","ACN","AGE","MONEY"],  # Your entity labels
    "colors": colors
}


def classify_and_visualize_ner(sentence):

    doc = nlp(sentence)

    displacy.render(doc, style="ent", options=options, jupyter=True)

sentence = "Priya Nair, prescribed Ibuprofen 400 mg for joint pain, Phone No.: +91 9988776654, Email: priya.nair@gmail.com, scheduled for surgery at Apollo Hospitals, Mumbai on 10th October 2023."


classify_and_visualize_ner(sentence)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.7.6 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed du